In [1]:
import os
import numpy as np
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import torch
from shark_turbine.aot import *
from iree.compiler.ir import Context
from datasets import load_dataset
import PIL

In [2]:
# Load feature extractor and pretrained model from huggingface
extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

#Do I need a state schema?? 

C:\Users\zjgar\code\SHARK-Turbine\examples\llama2_inference\llama2.venv\Lib\site-packages\transformers\models\convnext\feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [3]:
###
#standard inference example (no SHARK-Turbine):
###

#load an example
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

#image.save("cats-image.jpg") #if you want to see the cute picture :)

#image2 = PIL.JpegImagePlugin.JpegImageFile("yourexamplepicture.jpg")

In [4]:
#extract features from image(s) to feed to model
inputs = extractor(image, return_tensors="pt")
#inputs2 = extractor(image2, return_tensors="pt")
pixel_tensor = inputs.pixel_values

In [5]:
z= support.procedural.base.abstractify_single_value(pixel_tensor)
print(z)

AbstractTensor(1, 3, 224, 224, dtype=torch.float32)


In [6]:
#standard torch application of model to extracted feature input.

logits = model.forward(pixel_tensor).logits
#    logits2 = model(**inputs2).logits

predicted_label = logits.argmax(-1).item()
#predicted_label2 = logits2.argmax(-1).item()

print(model.config.id2label[predicted_label])
#print(model.config.id2label[predicted_label2])


tabby, tabby cat


In [8]:
###
#Using SHARK-Turbine to write a torch-mlir file for performing inference with the model.
###

#Questions: what is the minimum necessary functionality to include in a CompiledModule subclass? 

class wtfamidoing(CompiledModule):
    params = export_parameters(model, initialize =False) #         is this necessary? 

    @jittable
    def forward(pixels : torch.Tensor):
        logits0 = model.forward(pixels).logits
        label = logits0.argmax(-1).item()
        #str_output = model.config.id2label[predicted_label0] 
        return label

        '''def forward(self, x = AbstractTensor(1, 3, 224, 224, dtype=torch.float32) ):
        label = self.forward(x)
        return label'''



In [9]:
#build an instance of our CompiledModule and get the corresponding mlir code
inst = wtfamidoing(context=Context(), import_to="IMPORT")
module_str = str(CompiledModule.get_mlir_module(inst))

In [10]:
print(module_str)

module @wtfamidoing {
  util.global private @_params.resnet.embedder.embedder.convolution.weight {noinline} : tensor<64x3x7x7xf32>
  util.global private @_params.resnet.embedder.embedder.normalization.weight {noinline} : tensor<64xf32>
  util.global private @_params.resnet.embedder.embedder.normalization.bias {noinline} : tensor<64xf32>
  util.global private @_params.resnet.encoder.stages.0.layers.0.layer.0.convolution.weight {noinline} : tensor<64x64x3x3xf32>
  util.global private @_params.resnet.encoder.stages.0.layers.0.layer.0.normalization.weight {noinline} : tensor<64xf32>
  util.global private @_params.resnet.encoder.stages.0.layers.0.layer.0.normalization.bias {noinline} : tensor<64xf32>
  util.global private @_params.resnet.encoder.stages.0.layers.0.layer.1.convolution.weight {noinline} : tensor<64x64x3x3xf32>
  util.global private @_params.resnet.encoder.stages.0.layers.0.layer.1.normalization.weight {noinline} : tensor<64xf32>
  util.global private @_params.resnet.encoder.st

In [11]:
#Output a torch-ir mlir file
with open("resnet-18.mlir", "w+") as f:
    f.write(module_str)